# LangChain MCP Client Example

This notebook demonstrates how to use MultiServerMCPClient with LangChain agents.


In [ ]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from dotenv import load_dotenv

load_dotenv()


In [ ]:
async def setup_client():
    """Initialize MCP client with stock server."""
    client = MultiServerMCPClient(
        {
            "stock": {
                "command": "python",
                "args": ["D:\\AIML\\mcp learn2\\StockMcp\\mcp_server\\server.py"],
                "transport": "stdio",
            }
        }
    )
    tools = await client.get_tools()
    print(f"Loaded {len(tools)} tools: {[t.name for t in tools]}")
    return client, tools

client, tools = await setup_client()


In [ ]:
# Create agent with tools
agent = create_agent(
    "llama-3.1-8b-instant",  # or use your preferred model
    tools
)


In [ ]:
# Example: Get stock price
stock_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "What's the stock price of GOOGL?"}]}
)
print(stock_response)


In [ ]:
# # Example: Get stock news
# news_response = await agent.ainvoke(
#     {"messages": [{"role": "user", "content": "What's the latest news about Google stock?"}]}
# )
# print(news_response)


In [4]:
import asyncio
from email import message
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, START, END
from langchain.agents import create_agent
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict
from pydantic import BaseModel, Field, config
from typing import Annotated, List, Dict
import json

load_dotenv()

llm = ChatGroq(model="llama-3.3-70b-versatile")

class AgentState(TypedDict):
    messages : Annotated[list, add_messages]
    # prices : List
    # news : List


async def main():
    client = MultiServerMCPClient(
        {
            "stock": {
                "command": "python",
                "args": ["D:\\AIML\\mcp learn2\\StockMcp\\mcp_server\\server.py"],
                "transport": "stdio",
            }
        }
    )

    tools = await client.get_tools()
    agent = create_agent(llm, tools, state_schema=AgentState)

    # async def chatbot(state: AgentState) -> AgentState:
    #     print(state["messages"])
    #     response = await agent.ainvoke(state["messages"])
    #     return {"messages" : [response]}

    
    graph = StateGraph(AgentState)
    graph.add_node("chatbot", agent)
    graph.add_node("tools", ToolNode(tools))
    graph.add_conditional_edges("chatbot", tools_condition)
    graph.add_edge(START, "chatbot")
    graph.add_edge("tools", "chatbot")

    # memory = MemorySaver()
    app = graph.compile()
    # config = {"configurable":{"thread_id":"1"}}

    response = await app.ainvoke({"messages" : ["hi how are you"]})
    for m in response["messages"]:
        m.pretty_print()

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop